In [1]:
try:
    import os
    import sys
    
    import elasticsearch
    from elasticsearch import Elasticsearch 
    import pandas as pd
    from elasticsearch import helpers
    print("All Modules Loaded ! ")
except Exception as e:
    print("Some Modules are Missing {}".format(e))

All Modules Loaded ! 


In [2]:
def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
        print('Connected ')
    else:
        print('not connect-_-!')
    return es
es = connect_elasticsearch()

Connected 


In [25]:
#getting all index
res =  es.indices.get_alias("*")
for Name in res:
    print(Name)

ilm-history-3-000001
similarity-score
.kibana-event-log-7.10.1-000001
tutorial
maps
.apm-custom-link
.apm-agent-configuration
demo
.kibana_1
.kibana_task_manager_1


/home/shrey/DS/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.apm-custom-link, .apm-agent-configuration, .kibana_1, .kibana_task_manager_1], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [3]:
df= pd.read_csv("netflix_titles.csv")

In [4]:
 df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [30]:
df.isna().sum()

show_id            0
type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
listed_in          0
description        0
dtype: int64

In [31]:
df  = df.dropna()

In [32]:
 df.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [34]:
#convert in to elasticsearch format
df = df.to_dict('records')

In [35]:
df[0]

{'show_id': 81145628,
 'type': 'Movie',
 'title': 'Norm of the North: King Sized Adventure',
 'director': 'Richard Finn, Tim Maltby',
 'cast': 'Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson',
 'country': 'United States, India, South Korea, China',
 'date_added': 'September 9, 2019',
 'release_year': 2019,
 'rating': 'TV-PG',
 'duration': '90 min',
 'listed_in': 'Children & Family Movies, Comedies',
 'description': 'Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.'}

In [36]:
def generator(df2):
    for c,line in enumerate(df2):
        yield{
            '_index':'mynetflix',
            '_type':'_doc',
            '_id':line.get('show_id',None),
            '_source':{
                'title':line.get('title',""),
                'director':line.get('director',""),
                'description':line.get('description',""),
                'duration':line.get('duration',None),
            }
        }
    raise StopIteration

In [38]:
mycustom = generator(df)

In [39]:
mycustom

<generator object generator at 0x7f4edc714970>

In [40]:
next(mycustom)

{'_index': 'mynetflix',
 '_type': '_doc',
 '_id': 81145628,
 '_source': {'title': 'Norm of the North: King Sized Adventure',
  'director': 'Richard Finn, Tim Maltby',
  'description': 'Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.',
  'duration': '90 min'}}

In [43]:
#upload data
try:
    res = helpers.bulk(es, generator(df))
    print('uploades')
except Exception as e:
    pass

/home/shrey/DS/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


generator raised StopIteration


In [7]:
myquey = {
   "_source": [],
   "size": 10,
   "query": {
      "bool": {
         "must": [],
         "filter": [
            {
               "exists": {
                  "field": "description"
               }
            }
         ],
         "should": [
            {
               "match_phrase": {
                  "description": "movie "
               }
            }
         ],
         "must_not": []
      }
   }
}

res = es.search(
  index = 'mynetflix',
  scroll = '2m',
  size = 5,
  body = myquey)


In [8]:
res

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFmhPTktCVGNLU2JLUzdUTWNKbzlzVVEAAAAAAAAAAhZNM3RZS2ZuZ1EwR1dJWVBYX055aEdB',
 'took': 72,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3500, 'relation': 'eq'},
  'max_score': 5.2134967,
  'hits': [{'_index': 'mynetflix',
    '_type': '_doc',
    '_id': '70253171',
    '_score': 5.2134967,
    '_source': {'title': 'Jeff Dunham: Minding the Monsters',
     'director': 'Manny Rodriguez',
     'description': 'Jeff Dunham returns with a haunted house full of his hysterical entourage, and each character stars in its own horror movie trailer.',
     'duration': '80 min'}},
   {'_index': 'mynetflix',
    '_type': '_doc',
    '_id': '81028574',
    '_score': 5.1239834,
    '_source': {'title': 'Shopkins: Wild',
     'director': 'Adele K. Thomas, Richard Bailey',
     'description': 'Beloved movie star Scarletta Gateau visits Shopville and recruits the Shopkins